In [1]:
import pandas as pd
import pandasql as ps
import numpy as np

In [2]:
teams = pd.read_csv('./Team_nodes.csv')

In [3]:
teams.columns

Index(['postGame_side', 'postGame_goals_scored', 'postGame_goals_conceded',
       'postGame_index', 'postGame_total_shots', 'postGame_total_xg',
       'postGame_total_big_chances', 'postGame_total_key_passes',
       'competitionId', 'gameId', 'teamId', 'preGame_steph',
       'preGame_quality_overall', 'preGame_quality_attack',
       'preGame_quality_midfield', 'preGame_quality_defence',
       'preGame_proportion_pts', 'gameDate', 'teamName', 'competitionName',
       'nodeId', 'nodeName'],
      dtype='object')

In [29]:
query = """
    select *, strftime('%Y', gameDate) as year, strftime('%m', gameDate) as month
    from teams
    order by gameDate, teamName
"""
teams = ps.sqldf(query)

In [76]:
query = "select teamName,"
_columns = ['postGame_side',
       'competitionId', 'gameId', 'teamId', 'preGame_steph',
       'preGame_quality_overall', 'preGame_quality_attack',
       'preGame_quality_midfield', 'preGame_quality_defence',
       'preGame_proportion_pts', 'gameDate', 'teamName', 'competitionName',
       'nodeId', 'nodeName', 'year', 'month']
post_columns = ['postGame_goals_scored', 'postGame_goals_conceded',
       'postGame_index', 'postGame_total_shots', 'postGame_total_xg',
       'postGame_total_big_chances', 'postGame_total_key_passes']
for column in _columns:
    q_plus = f"""
        {column},
    """
    query += q_plus
for i, column in enumerate(post_columns):
    try:
        post_columns[i+1]
        sep = ',' 
    except IndexError:
        sep = ''
    q_plus = f"""
                {column},
                case avg({column}) over (
                    partition by teamName, year
                    rows between unbounded preceding
                    and current row
                ) - {column}
                    when 0 then (
                        select avg(innerT.{column})
                        from teams innerT
                        where innerT.year < outerT.year
                        and innerT.teamName = outerT.teamName
                    )
                    else avg({column}) over (
                            partition by teamName, year
                            rows between unbounded preceding
                            and 1 preceding
                        )
                end as agg_{column}{sep}   
            """
    query += q_plus

query += """
    from teams outerT
    order by gameDate, teamName
"""
features = ps.sqldf(query)
features.to_csv('team_nodes_plus_features.csv')

In [77]:
q = """
    select  teamName,
            year,
            postGame_goals_scored,
            case sum(postGame_goals_scored) over (
                    partition by teamName, year
                    rows between unbounded preceding
                    and current row
                ) - postGame_goals_scored
                when 0 then (
                    select sum(iT.postGame_goals_scored)
                    from teams iT
                    where iT.year < oT.year
                    and iT.teamName = oT.teamName
                )
                    
                else sum(postGame_goals_scored) over (
                    partition by teamName, year
                    rows between unbounded preceding
                    and 1 preceding
                )
            end as agg
            
            
    from teams oT
    where teamName = 'Real Madrid'
"""
ps.sqldf(q)

,teamName,year,postGame_goals_scored,agg
0,Real Madrid,2018,1,NaN
1,Real Madrid,2018,1,1.0
2,Real Madrid,2018,0,2.0
3,Real Madrid,2018,0,2.0
4,Real Madrid,2018,0,2.0
5,Real Madrid,2018,1,2.0
6,Real Madrid,2018,1,3.0
7,Real Madrid,2018,2,4.0
8,Real Madrid,2018,4,6.0
9,Real Madrid,2018,0,10.0
